# 1. 注意力机制基础

在深入学习Transformer之前，我们需要理解注意力机制（Attention Mechanism）的基本概念。注意力机制是Transformer架构的核心组件。

## 1.1 传统RNN的局限性

让我们先回顾一下传统RNN在序列处理中的问题：

1. **串行计算**：RNN必须按顺序处理序列，无法并行化
2. **长期依赖问题**：随着序列长度增加，早期信息容易丢失
3. **信息瓶颈**：所有信息都必须压缩到最后一个隐藏状态中

![RNN问题示意图](images/rnn_problems.png)

注意力机制的提出就是为了解决这些问题。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from torch.autograd import Variable

# 设置中文字体和图表样式
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")

print(f"PyTorch版本: {torch.__version__}")
print(f"是否有CUDA: {torch.cuda.is_available()}")

## 1.2 注意力机制的直观理解

注意力机制模拟了人类的注意力过程。当我们看一张图片或读一段文字时，我们会把注意力集中在最相关的部分。

**例子**：翻译句子 "The cat sat on the mat" 为中文时，当我们翻译"cat"这个词时，我们主要关注原句中的"cat"，而不是其他词。

注意力机制的核心思想：
- **查询（Query）**：我们想要关注什么
- **键（Key）**：用于匹配查询的索引
- **值（Value）**：实际的信息内容

## 1.3 数学公式

注意力机制的基本计算过程可以用以下公式表示：

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

其中：
- $Q$ 是查询矩阵 (queries)
- $K$ 是键矩阵 (keys)
- $V$ 是值矩阵 (values)
- $d_k$ 是键向量的维度
- $\sqrt{d_k}$ 是缩放因子，防止softmax饱和

In [ ]:
def simple_attention(query, key, value):
    """
    简单的注意力机制实现
    
    Args:
        query: 查询向量 [batch_size, seq_len, d_model]
        key: 键向量 [batch_size, seq_len, d_model]
        value: 值向量 [batch_size, seq_len, d_model]
    
    Returns:
        output: 注意力输出
        attention_weights: 注意力权重
    """
    # 计算注意力分数 (query与key的点积)
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / np.sqrt(d_k)
    
    # 应用softmax获得注意力权重
    attention_weights = F.softmax(scores, dim=-1)
    
    # 计算加权输出
    output = torch.matmul(attention_weights, value)
    
    return output, attention_weights

# 测试简单注意力机制
batch_size = 1
seq_len = 4
d_model = 6

# 创建示例输入
torch.manual_seed(42)
query = torch.randn(batch_size, seq_len, d_model)
key = torch.randn(batch_size, seq_len, d_model)
value = torch.randn(batch_size, seq_len, d_model)

output, attention_weights = simple_attention(query, key, value)

print(f"输入形状:")
print(f"Query: {query.shape}")
print(f"Key: {key.shape}")
print(f"Value: {value.shape}")
print(f"\n输出形状:")
print(f"Output: {output.shape}")
print(f"Attention weights: {attention_weights.shape}")

## 1.4 可视化注意力权重

让我们可视化注意力权重矩阵，理解不同位置之间的关注程度：

In [ ]:
def visualize_attention(attention_weights, title="注意力权重热力图"):
    """
    可视化注意力权重矩阵
    """
    # 取第一个batch的注意力权重
    weights = attention_weights[0].detach().numpy()
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(weights, annot=True, cmap='Blues', fmt='.3f',
                xticklabels=[f'位置{i}' for i in range(weights.shape[1])],
                yticklabels=[f'查询{i}' for i in range(weights.shape[0])])
    plt.title(title)
    plt.xlabel('Key位置')
    plt.ylabel('Query位置')
    plt.tight_layout()
    plt.show()

# 可视化之前计算的注意力权重
visualize_attention(attention_weights, "简单注意力机制权重可视化")

## 1.5 注意力机制的实际应用示例

让我们通过一个文本分类的例子来理解注意力机制的作用：

In [ ]:
class AttentionClassifier(nn.Module):
    """
    带有注意力机制的简单分类器
    """
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes):
        super(AttentionClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        
        # 注意力层
        self.attention = nn.Linear(hidden_dim, 1)
        
        # 分类层
        self.classifier = nn.Linear(hidden_dim, num_classes)
        
    def forward(self, x):
        # 词嵌入
        embedded = self.embedding(x)  # [batch_size, seq_len, embed_dim]
        
        # LSTM编码
        lstm_out, _ = self.lstm(embedded)  # [batch_size, seq_len, hidden_dim]
        
        # 计算注意力权重
        attention_scores = self.attention(lstm_out)  # [batch_size, seq_len, 1]
        attention_weights = F.softmax(attention_scores, dim=1)  # [batch_size, seq_len, 1]
        
        # 加权求和
        weighted_output = torch.sum(lstm_out * attention_weights, dim=1)  # [batch_size, hidden_dim]
        
        # 分类
        logits = self.classifier(weighted_output)
        
        return logits, attention_weights.squeeze(-1)

# 创建模型实例
vocab_size = 1000
embed_dim = 128
hidden_dim = 64
num_classes = 2

model = AttentionClassifier(vocab_size, embed_dim, hidden_dim, num_classes)

# 模拟一些数据
batch_size = 2
seq_len = 10
sample_input = torch.randint(0, vocab_size, (batch_size, seq_len))

# 前向传播
logits, attention_weights = model(sample_input)

print(f"输入形状: {sample_input.shape}")
print(f"输出logits形状: {logits.shape}")
print(f"注意力权重形状: {attention_weights.shape}")
print(f"\n每个样本的注意力权重和: {attention_weights.sum(dim=1)}")

In [ ]:
# 可视化分类器的注意力权重
def visualize_sequence_attention(attention_weights, sample_idx=0):
    """
    可视化序列中每个位置的注意力权重
    """
    weights = attention_weights[sample_idx].detach().numpy()
    positions = range(len(weights))
    
    plt.figure(figsize=(12, 4))
    bars = plt.bar(positions, weights, alpha=0.7, color='skyblue')
    plt.xlabel('序列位置')
    plt.ylabel('注意力权重')
    plt.title(f'样本 {sample_idx} 的注意力权重分布')
    plt.xticks(positions)
    
    # 在柱状图上添加数值
    for i, bar in enumerate(bars):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{weights[i]:.3f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

# 可视化两个样本的注意力权重
for i in range(min(2, batch_size)):
    visualize_sequence_attention(attention_weights, i)

## 1.6 注意力机制的优势

通过上面的例子，我们可以看到注意力机制的几个关键优势：

1. **并行计算**：不同位置的注意力可以同时计算
2. **长距离依赖**：任意两个位置之间的距离都是常数
3. **可解释性**：注意力权重提供了模型关注点的直观解释
4. **灵活性**：可以根据任务需求调整注意力的计算方式

### 与传统方法的对比：

| 特性 | RNN/LSTM | 注意力机制 |
|------|----------|------------|
| 计算复杂度 | O(n) | O(n²) |
| 并行性 | 串行 | 并行 |
| 长距离依赖 | 困难 | 容易 |
| 可解释性 | 低 | 高 |

## 1.7 练习和思考

1. **代码实践**：尝试修改上面的`simple_attention`函数，添加温度参数来控制注意力的尖锐程度

2. **理论思考**：为什么要除以$\sqrt{d_k}$？尝试不加这个缩放因子，观察结果有什么变化

3. **应用思考**：注意力机制除了在NLP中应用，还可以应用在哪些领域？

4. **实验扩展**：尝试在不同的数据集上应用带注意力的分类器，观察注意力权重的分布

In [ ]:
# 练习1：带温度参数的注意力机制
def attention_with_temperature(query, key, value, temperature=1.0):
    """
    带温度参数的注意力机制
    
    Args:
        temperature: 温度参数，控制注意力的尖锐程度
                    temperature > 1: 更平滑的注意力分布
                    temperature < 1: 更尖锐的注意力分布
    """
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / np.sqrt(d_k)
    
    # 应用温度参数
    scores = scores / temperature
    
    attention_weights = F.softmax(scores, dim=-1)
    output = torch.matmul(attention_weights, value)
    
    return output, attention_weights

# 比较不同温度下的注意力分布
temperatures = [0.5, 1.0, 2.0]
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, temp in enumerate(temperatures):
    _, weights = attention_with_temperature(query, key, value, temperature=temp)
    
    im = axes[i].imshow(weights[0].detach().numpy(), cmap='Blues')
    axes[i].set_title(f'温度 = {temp}')
    axes[i].set_xlabel('Key位置')
    axes[i].set_ylabel('Query位置')
    
    # 添加数值标注
    for row in range(weights.shape[1]):
        for col in range(weights.shape[2]):
            axes[i].text(col, row, f'{weights[0][row][col]:.2f}', 
                        ha='center', va='center', fontsize=8)

plt.tight_layout()
plt.show()

print("观察：")
print("- 温度 < 1：注意力更加集中，权重分布更尖锐")
print("- 温度 > 1：注意力更加分散，权重分布更平滑")
print("- 温度 = 1：标准的注意力机制")

## 总结

在这个教程中，我们学习了：

1. **注意力机制的动机**：解决RNN的串行计算和长期依赖问题
2. **核心概念**：Query、Key、Value的概念和计算过程
3. **数学原理**：注意力机制的数学公式
4. **实际应用**：如何在神经网络中使用注意力机制
5. **可视化技巧**：如何理解和分析注意力权重

注意力机制是理解Transformer的基础。在下一个教程中，我们将深入学习**自注意力机制（Self-Attention）**，这是Transformer架构的核心组件。

### 下一步学习：
- [02-self-attention.ipynb](02-self-attention.ipynb) - 自注意力机制详解